# Metody Obliczeniowe w Nauce i Technice Laboratorium 6
## Page searcher
### Paweł Gorgolewski

In [18]:
import os
import numpy as np
import pickle
import csv
import wikipedia as wiki
from typing import List, Dict, DefaultDict
from wikipedia.exceptions import WikipediaException
from collections import namedtuple, defaultdict
from functools import reduce
from itertools import chain
from collections import Counter
from time import time
from concurrent.futures import ProcessPoolExecutor


#### Generating wiki dump
Dump indeksów artykułów z wikipedii został pobrany poprzez link
https://dumps.wikimedia.org/enwiki/20220401/

Mając tytuły wszystkich artykuł, zaciągamy zawartość tytułu poprzez bilbiotekę *wikipedia*. Następnie zapisujemy pickle pythonowe w celu łatwego ładowania do programu

In [19]:
def get_first_n_titles(n=1500, file='data\wiki-pages-indexes.txt'):
    titles = []
    with open(file, "r", encoding='utf8') as f:
        csv_reader = csv.reader(f, delimiter=":")
        for row in csv_reader:
            if n < 0:
                break

            title = row[-1]
            if '/' in title:
                continue

            titles.append(title)
            n-=1

    return titles

def get_articles_content_and_save_pickle(titles: List[str]):
    not_matched = 0
    for title in titles:
        try:
            wiki_page = wiki.page(title)
            wiki_content = wiki_page.content
            #now saving pickle
            pickle.dump(wiki_content, open(f'data\\{title}', "wb"))
        except WikipediaException:
            not_matched += 1

    print(f"Not matched {not_matched} titles")

#titles = get_first_n_titles()
#get_articles_content_and_save_pickle(titles)

## Ładowanie artykułów

In [20]:
def get_articles(to_omit_file='wiki-pages-indexes.txt'):
    data_by_title = dict()
    for title in pickles():
        if title == to_omit_file or "dump" in title:
            continue

        data_by_title[title] = pickle.load(open(f"data\\{title}", "rb"))

    pickle.dump(data_by_title, open(f"data\\dumps\\articles_dict_dump_of_len_{len(data_by_title)}", "wb"))
    return data_by_title

def pickles(path='data'):
    for content in os.listdir(path):
        if os.path.isfile(os.path.join(path, content)):
            yield content

def get_articles_from_dump(dirpath="data\\dumps\\", dump_name="articles_dict_dump_of_len_1255"):
    path = os.path.join(dirpath, dump_name)
    return pickle.load(open(path, "rb"))

Używam wcześniej zapisanych struktur, aby nie tracić czasu na tworzenie ich.

In [21]:
#articles = get_articles()
articles = get_articles_from_dump()

## Tworzenie zbioru wszystkich słów

In [22]:
ArticleData = namedtuple('ArticleData', ['words_data', 'words_count', 'unique_words_count'])
class ArticlesParser:
    def __init__(self, articles: Dict[str, str]):
        self.articles = articles
        self.parsed_articles = dict()
        self.all_words_count = 0
        self.all_words_data = defaultdict(lambda: 0)

    def parse_article(self, content: str):
        all_words = content.split()
        words_data = defaultdict(lambda: 0)

        for word in all_words:
            words_data[word] += 1
            self.all_words_data[word] += 1

        self.all_words_count += len(all_words)
        return ArticleData(words_data=words_data, words_count=len(all_words), unique_words_count=len(words_data))

    def parse_articles(self):
        for article in articles:
            self.parsed_articles[article] = self.parse_article(articles[article])


parser = ArticlesParser(articles)
parser.parse_articles()
print(parser.all_words_count)
print(parser.all_words_data['A'])


4898002
5540
